In [2]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [6]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import numpy as np
from numba import jit

In [7]:
pickle_file = '/Users/shivankurkapoor/Desktop/Study/ML/Udacity/Assignment1/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [8]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

@jit
def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [9]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            /predictions.shape[0])

# Calculating Output Size
O=(W−K−2P)/S +1  
O is the output height/length  
W is the input height/length  
K is the filter size  
P is the padding  
"same" = -1  
"valid" = 0  
S is the stride

In [14]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()
with graph.as_default():
    #Input data
    tf_train_dataset = tf.placeholder(
        tf.float32, shape = (batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #Variables (in,in,in,out)
    
    #Covolution Layer 1
    layer1_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros(depth))
    
    #Convolution Layer 2
    layer2_weights = tf.Variable(tf.truncated_normal(
        [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    #Fully Connected Layer 
    #Based on above convolutions, out image dimensions will be reduced by a factor of 4
    layer3_weights = tf.Variable(tf.truncated_normal(
        [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    #Softmax Layer
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    
    #Model
    def model(data):
        #Conv Layer 1
        conv = tf.nn.conv2d(data, layer1_weights, [1,2,2,1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        
        #Conv Layer 2
        conv = tf.nn.conv2d(hidden, layer2_weights, [1,2,2,1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        
        #Fully Connected Layer
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        
        #Softmax Layer
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    #Training
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    #Predictions
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))
    
    
    

In [15]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.419519
Minibatch accuracy: 6.2%
Validation accuracy: 9.1%
Minibatch loss at step 50: 2.071228
Minibatch accuracy: 25.0%
Validation accuracy: 37.9%
Minibatch loss at step 100: 1.076032
Minibatch accuracy: 68.8%
Validation accuracy: 70.1%
Minibatch loss at step 150: 0.722506
Minibatch accuracy: 87.5%
Validation accuracy: 75.6%
Minibatch loss at step 200: 0.837805
Minibatch accuracy: 81.2%
Validation accuracy: 78.3%
Minibatch loss at step 250: 1.250257
Minibatch accuracy: 68.8%
Validation accuracy: 78.0%
Minibatch loss at step 300: 0.520260
Minibatch accuracy: 81.2%
Validation accuracy: 78.8%
Minibatch loss at step 350: 0.988435
Minibatch accuracy: 81.2%
Validation accuracy: 76.6%
Minibatch loss at step 400: 0.281744
Minibatch accuracy: 87.5%
Validation accuracy: 80.0%
Minibatch loss at step 450: 0.798771
Minibatch accuracy: 75.0%
Validation accuracy: 78.2%
Minibatch loss at step 500: 0.676622
Minibatch accuracy: 81.2%
Validation accuracy: 80.6%
Min

In [16]:
#Problem 1

In [22]:
# Create image size function based on input, filter size, padding and stride
# 2 convolutions only
def output_size_no_pool(input_size, filter_size, padding, conv_stride):
    if padding == 'same':
        padding = -1.00
    elif padding == 'valid':
        padding = 0.00
    else:
        return None
    output_1 = float(((input_size - filter_size - 2*padding) / conv_stride) + 1.00)
    output_2 = float(((output_1 - filter_size - 2*padding) / conv_stride) + 1.00)
    return int(np.ceil(output_2))

patch_size = 5
final_image_size = output_size_no_pool(image_size, patch_size, padding='same', conv_stride=2)
print(final_image_size)

7


In [23]:
batch_size = 16
depth  =16
num_hidden = 64
graph = tf.Graph()

with graph.as_default():
    #Input data
    tf_train_dataset = tf.placeholder(
        tf.float32, shape = (batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    '''
    Variables
    '''
    # Conv layer 1
    layer1_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    # Conv Layer 2
    layer2_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    #Fully Connected Layer
    # Fully Connected Layer (Densely Connected Layer)
    # Use neurons to allow processing of entire image
    layer3_weights = tf.Variable(tf.truncated_normal(
            [final_image_size * final_image_size * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    #Softmax Layer
    layer4_weights = tf.Variable(tf.truncated_normal(
            [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    
    '''
    Model
    '''
    def model(data):
        #First Conv Layer with Pooling
        conv_1 = tf.nn.conv2d(data, layer1_weights, strides=[1,1,1,1], padding = 'SAME')
        hidden_1 = tf.nn.relu(conv_1 + layer1_biases)
        pool_1 = tf.nn.max_pool(hidden_1, [1,2,2,1],[1,2,2,1], padding='SAME')
        
        #Second Conv Layer with Pooling
        conv_2 = tf.nn.conv2d(pool_1, layer2_weights, strides=[1,1,1,1], padding = 'SAME')
        hidden_2 = tf.nn.relu(conv_2 + layer2_biases)
        pool_2 = tf.nn.max_pool(hidden_2, [1,2,2,1],[1,2,2,1], padding='SAME')
        
        #Fully Connected LAyer
        shape = pool_2.get_shape().as_list()
        reshape = tf.reshape(pool_2, [shape[0], shape[1]*shape[2]*shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        
        #Softmax Layer
        return tf.matmul(hidden, layer4_weights) + layer4_biases
    
    '''
    Training
    '''
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
            labels=tf_train_labels, logits=logits))
    
    '''
    Optimizer
    '''
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    '''
    Predictions
    '''
    train_prediction = tf.nn.softmax(logits) 
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))
    
    
    
    

In [26]:
num_steps = 30000

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.014224
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 500: 0.705757
Minibatch accuracy: 75.0%
Validation accuracy: 81.4%
Minibatch loss at step 1000: 0.444069
Minibatch accuracy: 81.2%
Validation accuracy: 84.2%
Minibatch loss at step 1500: 0.587773
Minibatch accuracy: 81.2%
Validation accuracy: 85.2%
Minibatch loss at step 2000: 0.056074
Minibatch accuracy: 100.0%
Validation accuracy: 85.5%
Minibatch loss at step 2500: 0.615014
Minibatch accuracy: 87.5%
Validation accuracy: 85.7%
Minibatch loss at step 3000: 0.724009
Minibatch accuracy: 81.2%
Validation accuracy: 86.8%
Minibatch loss at step 3500: 0.445266
Minibatch accuracy: 75.0%
Validation accuracy: 86.4%
Minibatch loss at step 4000: 0.382132
Minibatch accuracy: 87.5%
Validation accuracy: 87.3%
Minibatch loss at step 4500: 0.529550
Minibatch accuracy: 81.2%
Validation accuracy: 87.7%
Minibatch loss at step 5000: 0.852780
Minibatch accuracy: 75.0%
Validation accura

In [ ]:
#Problem 2 - Implemeting LeNet5

# Details
image_size = 28  
### Convolutions  
conv_filter_size = 5  
conv_stride = 1  
### Average Pooling  
pool_filter_size = 2  
pool_stride = 2  
padding='valid'  
### Prevent overfitting  
Learning rate decay  
Regularization  
Dropout  
### Layers  
Convolution  
Pooling  
Convolution  
Pooling  
Fully-connected  
Fully-connected  
Readout  

In [27]:
image_size = 28
# Create image size function based on input, filter size, padding and stride
# 2 convolutions only with 2 pooling
def output_size_pool(input_size, conv_filter_size, pool_filter_size, padding, conv_stride, pool_stride):
    if padding == 'same':
        padding = -1.00
    elif padding == 'valid':
        padding = 0.00
    else:
        return None
    # After convolution 1
    output_1 = (((input_size - conv_filter_size - 2*padding) / conv_stride) + 1.00)
    # After pool 1
    output_2 = (((output_1 - pool_filter_size - 2*padding) / pool_stride) + 1.00)    
    # After convolution 2
    output_3 = (((output_2 - conv_filter_size - 2*padding) / conv_stride) + 1.00)
    # After pool 2
    output_4 = (((output_3 - pool_filter_size - 2*padding) / pool_stride) + 1.00)  
    return int(output_4)

final_image_size = output_size_pool(input_size=image_size, conv_filter_size=5, pool_filter_size=2, padding='valid', conv_stride=1, pool_stride=2)
print(final_image_size)

4


In [31]:
batch_size = 16
depth = 32
num_hidden = 64
beta = 0.001

graph = tf.Graph()

with graph.as_default():
    '''
    Input Data
    '''
    #in = 1 out = 16
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, 
                                                         image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, 
                                                        num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    '''
    Variables
    '''
    #Conv layer 1
    layer1_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    #Conv layer 2
    #in = 16 out = 16
    layer2_weights = tf.Variable(tf.truncated_normal(
            [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    #Fully connected layer 1
    final_image_size = output_size_pool(
        input_size=image_size, 
        conv_filter_size=5, 
        pool_filter_size=2, 
        padding='valid', 
        conv_stride=1, 
        pool_stride=2)
    
    layer3_weights = tf.Variable(tf.truncated_normal(
            [final_image_size * final_image_size * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    #Fully connected layer 2
    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_hidden], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape = [num_hidden]))
    
    #Softmax Layer
    layer5_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.01))
    layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    '''
    Model
    '''
    def model(data):
        # 1. conv layer + avg pooling
        conv_1 = tf.nn.conv2d(data, layer1_weights, strides=[1,1,1,1], padding='VALID')
        hidden_1 = tf.nn.relu(conv_1 + layer1_biases)
        pool_1 = tf.nn.avg_pool(hidden_1, [1,2,2,1], [1,2,2,1], padding='VALID')
        
        # 2. conv layer + avg pooling
        conv_2 = tf.nn.conv2d(pool_1, layer2_weights, strides=[1,1,1,1], padding='VALID')
        hidden_2 = tf.nn.relu(conv_2 + layer2_biases)
        pool_2 = tf.nn.avg_pool(hidden_2, [1,2,2,1], [1,2,2,1], padding='VALID')
        
        # 3. fully connected layer + dropout
        shape = pool_2.get_shape().as_list()
        reshape = tf.reshape(pool_2, [shape[0], shape[1]*shape[2]*shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        keep_prob = 0.5
        hidden_drop = tf.nn.dropout(hidden, keep_prob)
        
        # 4. fully connected layer + dropout
        hidden_2 = tf.nn.relu(tf.matmul(hidden_drop, layer4_weights) + layer4_biases)
        hidden_2_drop = tf.nn.dropout(hidden_2, keep_prob)
        
        #Softmax
        return tf.matmul(hidden_2_drop, layer5_weights) + layer5_biases
    
    '''
    Training
    '''
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
            labels=tf_train_labels, logits=logits))
    
    # Loss function with L2 Regularization 
    # regularizers = tf.nn.l2_loss(layer4_weights) + \
                   # tf.nn.l2_loss(layer5_weights)
    # loss = tf.reduce_mean(loss + beta * regularizers)
    
    '''
    Optimizer
    '''
    #Decay learning rate
    global_step = tf.Variable(0)
    start_learning_rate = 0.05
    #decayed_learning_rate = learning_rate *
    #                      decay_rate ^ (global_step / decay_steps)
    learning_rate = tf.train.exponential_decay(
        start_learning_rate, global_step, 100000, 0.96, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    '''
    Predictions
    '''
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))
    

In [32]:
num_steps = 30000

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.371780
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Minibatch loss at step 500: 1.188885
Minibatch accuracy: 68.8%
Validation accuracy: 58.0%
Minibatch loss at step 1000: 1.097084
Minibatch accuracy: 68.8%
Validation accuracy: 67.9%
Minibatch loss at step 1500: 1.552269
Minibatch accuracy: 68.8%
Validation accuracy: 72.5%
Minibatch loss at step 2000: 0.406173
Minibatch accuracy: 93.8%
Validation accuracy: 74.8%
Minibatch loss at step 2500: 0.907509
Minibatch accuracy: 75.0%
Validation accuracy: 76.9%
Minibatch loss at step 3000: 0.876250
Minibatch accuracy: 81.2%
Validation accuracy: 76.8%
Minibatch loss at step 3500: 0.742346
Minibatch accuracy: 75.0%
Validation accuracy: 78.3%
Minibatch loss at step 4000: 0.499270
Minibatch accuracy: 81.2%
Validation accuracy: 80.2%
Minibatch loss at step 4500: 1.026051
Minibatch accuracy: 68.8%
Validation accuracy: 80.6%
Minibatch loss at step 5000: 1.427529
Minibatch accuracy: 62.5%
Validation accuracy